### ***Logic***

##### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
##### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
     * Không quá 72h.
##### Backlog lấy
##### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
     * HCM -> HN: 12h.
     * Các tỉnh còn lại: 32h.
##### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
     * Giao hàng tại HCM HN: 36h.
     * Giao hàng tại các tỉnh còn lại: 56h.

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiKien',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery = data.loc[data['TrangThai'].isin([
    'Đang giao hàng', 'Giao không thành công',
    'Chờ xác nhận giao lại'
]) | ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] == data['KhoHienTai'])) |\
                          ((data['TrangThai'] == 'Tạo thành công') & ((~data['ThoiGianKetThucLay'].isnull()) ^\
                          (~data['ThoiGianKetThucGiao'].isnull())))]
delivery['LoaiBacklog'] = 'Xử lý giao'
internal_delivery = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery, 'N0'] = delivery.loc[internal_delivery]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery, 'N0'] = delivery.loc[~internal_delivery]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned = data.loc[data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | ((data['TrangThai'] == 'Trả hàng') & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai'])))) |\
          ((data['TrangThai'] == 'Tạo thành công') & (~data['ThoiGianKetThucGiao'].isnull()))]
returned['LoaiBacklog'] = 'Xử lý trả'
internal_return = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return, 'N0'] = returned.loc[internal_return]['ThoiGianKetThucGiao']
returned.loc[~internal_return, 'N0'] = returned.loc[~internal_return]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup = data[(data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))]
pickup['LoaiBacklog'] = 'Xử lý lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
transporting = data.loc[(data['TrangThai'] == 'GHN đã lấy') |\
                         ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] != data['KhoHienTai']))]
transporting['LoaiBacklog'] = 'Xử lý luân chuyển'
transporting_filter = (transporting['DenTinh'] == 'Hà Nội') | (transporting['DenTinh'] == 'Hồ Chí Minh')
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting.loc[transporting_filter, 'N+'] = transporting.loc[transporting_filter]['N0']+ pd.Timedelta(hours=12)
transporting.loc[~transporting_filter, 'N+'] = transporting.loc[~transporting_filter]['N0'] + pd.Timedelta(hours=32)
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
r_transporting = data.loc[data['TrangThai'].isin([
    'Trả hàng', 'Chờ trả hàng'
]) & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) |\
      ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai'])))]
r_transporting['LoaiBacklog'] = 'Xử lý luân chuyển trả'
r_transporting_filter = (r_transporting['DenTinh'] == 'Hà Nội') | (r_transporting['DenTinh'] == 'Hồ Chí Minh')
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting.loc[r_transporting_filter, 'N+'] = r_transporting.loc[r_transporting_filter]['N0'] + pd.Timedelta(hours=36)
r_transporting.loc[~r_transporting_filter, 'N+'] = r_transporting.loc[~r_transporting_filter]['N0'] + pd.Timedelta(hours=56)
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# convert Aging to hours
data['Aging (hours)'] = data['Aging']/np.timedelta64(1, 'h')
# set e-commerces column to NaN
data['e-commerces'] = np.nan


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'e-commerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'e-commerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'e-commerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'e-commerces'] = 'Lazada'


# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
# reset the index of dataframe
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data
# data['KhoGui']

In [11]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging (hours)',
    'e-commerces',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_everyday'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(hours=0)]

# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_everyday' id:181055436>

In [12]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val

In [13]:
# to excel
# data_to_excel = data[[
#     'MaDH',
#     'KhoHienTai',
#     'TrangThai',
#     'GhiChuGHN',
#     'SoLanLay',
#     'SoLanGiao',
#     'SoLanTra',
#     'LoaiBacklog',
#     'N0',
#     'N+',
#     'Aging',
#     'Aging (hours)',
#     'e-commerces',
# ]]

# data_to_excel.to_excel('backlog.excel', sheet='Sheet1')